In [1]:
import numpy as np
import pandas as pd
from numpy import unique
from numpy import where

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

import matplotlib.pyplot as plt

In [2]:
# Load data
data = pd.read_csv("data/data.csv",  header=None)
target = pd.read_csv("data/id.csv", header=None)

In [3]:
#print(data.head)
print("INFO data")
print(data.info(verbose=True))
print("INFO target")
print(target.info(verbose=True))

INFO data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 128 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64


In [4]:
# Data train into numpy
X = np.array(data.values)
print("INFO data train ndim ", X.ndim)
print("INFO data train shape ", X.shape)
# Data target into numpy
y = np.array(target.values)
y = le.fit_transform(y)
print("INFO data target ndim ", y.ndim)
print("INFO data target shape ", y.shape)

INFO data train ndim  2
INFO data train shape  (2450, 128)
INFO data target ndim  1
INFO data target shape  (2450,)


In [6]:
# Get indexes of all labels

samet_index = data[target[0]==0].index.tolist()
meyra_index = data[target[0]==1].index.tolist()
bekir_index = data[target[0]==2].index.tolist()
samir_index = data[target[0]==3].index.tolist()
berat_index = data[target[0]==4].index.tolist()
ramazan_index = data[target[0]==5].index.tolist()

# Show first 10 indexes
print("Bekir indexes ", bekir_index[:10])
print("Berat indexes ", berat_index[:10])
print("Meyra indexes ", meyra_index[:10])
print("Ramazan indexes ", ramazan_index[:10])
print("Samet indexes ", samet_index[:10])
print("Samir indexes ", samir_index[:10])

Bekir indexes  [934, 935, 936, 937, 938, 939, 940, 941, 942, 943]
Berat indexes  [1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725]
Meyra indexes  [479, 480, 481, 482, 483, 484, 485, 486, 487, 488]
Ramazan indexes  [2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258]
Samet indexes  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Samir indexes  [1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394]


In [7]:
# Get all indexes together
all_indexes = np.concatenate([bekir_index[:8],
                                berat_index[:8],
                                meyra_index[:8],
                                ramazan_index[:8],
                                samet_index[:8],
                                samir_index[:8]])
# all_indexes = np.vstack(all_indexes,
print(all_indexes)

[ 934  935  936  937  938  939  940  941 1716 1717 1718 1719 1720 1721
 1722 1723  479  480  481  482  483  484  485  486 2249 2250 2251 2252
 2253 2254 2255 2256    0    1    2    3    4    5    6    7 1385 1386
 1387 1388 1389 1390 1391 1392]


In [11]:
# get data and labels of all indexes
x_ten = data.loc[ all_indexes, : ]
y_ten = target.loc[ all_indexes , : ]

print("x_ten", x_ten)
print("y_ten", y)

x_ten            0         1         2         3         4         5         6    \
934  -0.141236  0.182262  0.056664  0.152511  0.083062  0.023147 -0.092564   
935  -0.172426  0.129670 -0.045568  0.080295  0.105105 -0.016185 -0.154171   
936  -0.128303  0.167794 -0.004384  0.085313  0.171333  0.047267 -0.121825   
937  -0.137400  0.129731 -0.022547  0.082919  0.111654  0.082956 -0.093362   
938  -0.110589  0.173985  0.081496  0.163161  0.123259  0.008574 -0.142290   
939  -0.041491  0.110605  0.031581  0.142755  0.098926  0.053113 -0.075830   
940  -0.133522  0.133000  0.036692  0.122378  0.095516  0.073730 -0.102879   
941  -0.091072  0.167259  0.022736  0.111351  0.152868  0.043894 -0.087147   
1716 -0.151765 -0.011192 -0.142109  0.049688  0.227407  0.167846 -0.020472   
1717 -0.124634 -0.072246 -0.201699  0.072524  0.216919  0.170886  0.012949   
1718 -0.105080 -0.089958 -0.210211  0.052916  0.201809  0.093275 -0.049499   
1719 -0.133819 -0.056217 -0.181862  0.052099  0.196044  0.

In [12]:
# Real test data
X_train, X_test, y_train, y_test = train_test_split(x_ten, y_ten, test_size=0.4, random_state=42)

In [13]:
# Supervised SVC parameter grid search
n_samples = len(X_train)
print("Train dat lenght ",n_samples)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['poly'], 'degree': [1, 3, 5, 7]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

Train dat lenght  28
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.167 (+/-0.533) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.167 (+/-0.533) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.167 (+/-0.533) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.167 (+/-0.533) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.642 (+/-0.142) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.167 (+/-0.533) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
1.000 (+/-0.000) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.642 (+/-0.142) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
1.000 (+/-0.000) for {'C': 1, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 10, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 100, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 1000, 'kernel': 'linear'}
1.000 (+/-0.000) for {'degree': 1, 'kernel': 'poly'}
1.000 (+/-0.000) for {'degree': 3